In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

## Validation Strategies

In [2]:
#Objective Functions to minimize with Bayesian Optimization

def SSV(x):
    # Single Shot Validation
    
    global rho, sigma_in, tikh_opt, k, ti, noise_opt

    #setting and initializing
    rho      = x[0]
    sigma_in = round(10**x[1],2)
    #ti       = timeit.timeit()
    ti       = time.time()
    lenn     = tikh.size
    Mean     = np.zeros(lenn)

    #Train using tv: training+val
    #Wout = train_n(U_washout, U_tv, Y_tv, tikh, sigma_in, rho, N_units)[0]
    Wout = train_n(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)[0]
    
    #t1   = timeit.timeit()
    t1       = time.time()
    
    #select washout and validation
    p      = N_in
    Y_val  = U[N_washout + p : N_washout + p + N_val].copy()
    U_wash = U[            p : N_washout + p        ].copy()

    #washout before closed loop
    xf = open_loop(U_wash, np.zeros(N_units), sigma_in, rho)[-1]
    
    for j in range(lenn):
        #Validate
        Yh_val   = closed_loop(N_val-1, xf, Wout[j], sigma_in, rho)[0]
        Mean[j] += np.log10(np.mean((Y_val-Yh_val)**2))
    
        # prevent from diverging to infinity: put MSE equal to 10^10
        if np.isnan(Mean[j]):
            Mean[j] = 10
        
    #if k==0: print('closed-loop time:', timeit.timeit() - t1)
    if k==0: print('closed-loop time:', time.time() - t1)

    #select optimal tikh
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1

    return Mean[a]


def KFC_Noise(x):
    #K-fold Validation
    
    global rho, sigma_in, tikh_opt, k, ti, noise_opt
    
    #setting and initializing
    rho      = x[0]
    sigma_in = round(10**x[1],2)    
    ti       = time.time()
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Wout, LHS0, RHS0 = train_n(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)
        
    #Different Folds in the validation set
    t1   = time.time()
    for i in range(N_fo):

       #select washout and validation
        p      = N_in + i*N_fw
        Y_val  = U[N_washout + p    : N_washout + p + N_val    ].copy()
        U_wash = U[            p    : N_washout + p            ].copy()
        
        #washout
        xf     = open_loop(U_wash, np.zeros(N_units), sigma_in, rho)[-1]
        #Train: remove the validation interval
        Xt     = open_loop(Y_val, xf[:N_units], sigma_in, rho)[:-1]
        
        LHS    = LHS0 - np.dot(Xt.T, Xt)
        RHS    = RHS0 - np.dot(Xt.T, Y_val)

        for j in range(lenn):
            if j == 0: #add tikhonov to the diagonal (fast way that requires less memory)
                LHS.ravel()[::LHS.shape[1]+1] += tikh[j]
            else:
                LHS.ravel()[::LHS.shape[1]+1] += tikh[j] - tikh[j-1]
            
            Wout[j]  = np.linalg.solve(LHS, RHS)

            #Validate
            Yh_val   = closed_loop(N_val-1, xf, Wout[j], sigma_in, rho)[0]
            Mean[j] += np.log10(np.mean((Y_val-Yh_val)**2))
        
                
    if k==0: print('closed-loop time:', time.time() - t1)
    
    #select optimal tikh
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1

    return Mean[a]/N_fo

def RVC_Noise(x):
    #Recycle Validation
    
    global rho, sigma_in, tikh_opt, k, ti, noise_opt
    
    #setting and initializing
    rho      = x[0]
    sigma_in = round(10**x[1],2)    
    ti       = time.time()
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Wout = train_n(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)[0]

    #Different Folds in the validation set
    t1   = time.time()
    for i in range(N_fo):
        
        #select washout and validation
        p      = N_in + i*N_fw
        Y_val  = U[N_washout + p : N_washout + p + N_val].copy()
        U_wash = U[            p : N_washout + p        ].copy()
        
        #washout before closed loop
        xf = open_loop(U_wash, np.zeros(N_units), sigma_in, rho)[-1]
                  
        for j in range(lenn):
            #Validate
            Yh_val   = closed_loop(N_val-1, xf, Wout[j], sigma_in, rho)[0]
            Mean[j] += np.log10(np.mean((Y_val-Yh_val)**2))
                            
    if k==0: print('closed-loop time:', time.time() - t1)
    
    #select optimal tikh
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1

    return Mean[a]/N_fo

def RVC_Noise_PH(x):
    #Recycle Validation with Prediction Horizon
    
    global rho, sigma_in, tikh_opt, k, ti, noise_opt
    
    #setting and initializing
    rho      = x[0]
    sigma_in = round(10**x[1],2)    
    ti       = time.time()
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Wout = train_n(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)[0]

    #Different Folds in the validation set
    t1   = time.time()
    for i in range(N_fo):
        
        #select washout and validation
        p      = N_in + i*N_fw
        Y_val  = U[N_washout + p : N_washout + p + N_val].copy()
        U_wash = U[            p : N_washout + p        ].copy()
        
        #washout before closed loop
        xf = open_loop(U_wash, np.zeros(N_units), sigma_in, rho)[-1]
                  
        for j in range(lenn):
            #Validate on cloop
            Yh_val   = closed_loop(N_val-1, xf, Wout[j], sigma_in, rho)[0]
            #Error with respect to data
            Y_err    = np.linalg.norm((Yh_val-Y_val),axis=1)/sigma_ph
            #Time for which the error crosses threshold
            PH       = np.argmax(Y_err>threshold_ph)
            if PH == 0 and Y_err[0]<threshold_ph: PH=N_val
            #Update Mean for minimization problem (minus sign)
            Mean[j] += -PH
                            
    if k==0: print('closed-loop time:', time.time() - t1)
    
    #select optimal tikh
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1

    return Mean[a]/N_fo/N_lyap
    
#     #Different Folds in the validation set
#     for i in range(N_fo):

#         p      = N_in + i*N_fw
#         Y_val  = U[N_washout + p : N_washout + p + N_val].copy()
        
#         for j in range(lenn):

#             #Validate
#             Yh_val      = closed_loop(N_val-1, Xa_train[p-1], Wout[j], sigma_in, rho)[0]
#             Y_err       = np.linalg.norm((Yh_val-Y_val),axis=1)/sigma_ph
#             PH          = np.argmax(Y_err>threshold_ph)
#             if PH == 0 and Y_err[0]<threshold_ph: PH=N_val
#             Mean[j] += -PH 
                
#     a           = np.argmin(Mean)
#     tikh_opt[k] = tikh[a]
#     k          +=1
#     print(k,'Par :', rho,sigma_in, tikh[a], Mean[a]/N_fo/N_lyap)
#     print('')


#     return Mean[a]/N_fo/N_lyap    